In [ ]:
import onnx
import json
from datetime import datetime

# Load ONNX model
model_path = "model.onnx"  # path to your model
onnx_model = onnx.load(model_path)

# Optionally check model
onnx.checker.check_model(onnx_model)

# Define STAC template
stac_item = {
    "stac_version": "1.0.0",
    "stac_extensions": [
        "https://stac-extensions.github.io/mlm/v1.4.0/schema.json",
        "https://stac-extensions.github.io/raster/v1.1.0/schema.json"
    ],
    "type": "Feature",
    "id": "geo_multisource_embedder_v1",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [-180.0, -90.0],
                [-180.0,  90.0],
                [ 180.0,  90.0],
                [ 180.0, -90.0],
                [-180.0, -90.0]
            ]
        ]
    },
    "bbox": [-180.0, -90.0, 180.0, 90.0],
    "properties": {
        "description": "Torch model ingesting Sentinel-2 L2A/L1C and Sentinel-1 GRD/RTC patches (224×224) and emitting geo-aware embeddings.",
        "datetime": "1900-01-01T00:00:00Z",
        "mlm:name": "GeoMultiSourceEmbedder",
        "mlm:tasks": ["classification", "embedding"],
        "mlm:architecture": "ViT-Hybrid-224",
        "mlm:framework": "onnx",
        "mlm:framework_version": onnx.__version__,
        "mlm:accelerator": "cuda", 
        "mlm:input": [
            {
                "name": "S2L2A patched reflectance",
                "constellation": "sentinel-2",
                "processing_level": "L2A",
                "bands": ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12"],
                "input": {
                    "shape": [-1, 12, 224, 224],
                    "dim_order": ["batch", "channel", "y", "x"],
                    "data_type": "float32"
                },
                "value_scaling": [{ "type": "min-max", "min": 0, "max": 1 }],
                "resize_type": None,
                "pre_processing_function": None
            },
            {
                "name": "S2L1C TOA reflectance",
                "constellation": "sentinel-2",
                "processing_level": "L1C",
                "bands": ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                "input": {
                    "shape": [-1, 13, 224, 224],
                    "dim_order": ["batch", "channel", "y", "x"],
                    "data_type": "float32"
                },
                "value_scaling": [{ "type": "min-max", "min": 0, "max": 1 }],
                "resize_type": None,
                "pre_processing_function": None
            },
            {
                "name": "S1GRD/RTC backscatter",
                "constellation": "sentinel-1",
                "processing_level": "GRD/RTC",
                "bands": ["VV","VH"],
                "input": {
                    "shape": [-1, 2, 224, 224],
                    "dim_order": ["batch", "channel", "y", "x"],
                    "data_type": "float32"
                },
                "value_scaling": [{ "type": "z-score", "mean": 0, "stddev": 1 }],
                "resize_type": None,
                "pre_processing_function": None
            }
        ],
        "mlm:output": [
            {
                "name": "geo-aware embeddings",
                "tasks": ["classification", "embedding"],
                "result": {
                    "shape": [24, 196, 1024],
                    "dim_order": ["batch", "y", "x", "embedding"],
                    "data_type": "float32"
                },
                "post_processing_function": None
            }
        ]
    },
    "assets": {
        "weights": {
            "href": "https://drive.google.com/file/d/1nImq7n_awDCKHTH4XMROvS3_4kRr28D2/view?usp=sharing",
            "title": "ONNX Weights",
            "description": "Inference-ready ONNX model",
            "type": "application/octet-stream; application=onnx",
            "roles": ["mlm:model", "mlm:weights"],
            "mlm:artifact_type": "onnx"
        },
        "readme": {
            "href": "https://huggingface.co/ibm-esa-geospatial/TerraMind-1.0-base",
            "title": "Model Card",
            "type": "text/markdown",
            "roles": ["documentation"]
        }
    },
    "links": []
}

# Save to file
with open("mlm_stac.json", "w") as f:
    json.dump(stac_item, f, indent=2)
